In [1]:
import torch
from torch import nn

In [2]:
def corr2d(x, k):
    """compute 2d cross-correlation."""
    h, w = k.shape
    y = torch.zeros((x.shape[0] - h + 1, x.shape[1] - w + 1))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (x[i:i+h, j:j+w]*k).sum()
    return y

In [6]:
x = torch.Tensor([[0,1,2],[3,4,5],[6,7,8]])
k = torch.Tensor([[0,1],[2,3]])
corr2d(x,k)

tensor([[19., 25.],
        [37., 43.]])

In [7]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeors(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [9]:
x = torch.ones(6,8)
x[:,2:6]=0
x

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [10]:
k = torch.tensor([[-1.,1.]])
y = corr2d(x,k)
y

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [11]:
corr2d(x.t(),k)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [14]:
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

x = x.reshape(1, 1, 6, 8)
y = y.reshape(1, 1, 6, 7)
lr = 3e-2

for i in range(10):
    y_hat = conv2d(x)
    l = (y_hat - y)**2
    conv2d.zero_grad()
    l.sum().backward()
    # update kernal
    conv2d.weight.data[:] -= lr*conv2d.weight.grad
    if (i+1) % 2 == 0:
        print(f'epoch {i+1} loss: {l.sum().item()}')

epoch 2 loss: 13.615275382995605
epoch 4 loss: 2.3015711307525635
epoch 6 loss: 0.3932284712791443
epoch 8 loss: 0.06887634843587875
epoch 10 loss: 0.012744824402034283


/Users/tangyun/opt/miniconda3/envs/rl/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [15]:
conv2d.weight.data

tensor([[[[-0.9855,  0.9762]]]])

In [16]:
conv2d(x)

tensor([[[[-0.0093, -0.9855,  0.0000,  0.0000,  0.0000,  0.9762, -0.0093],
          [-0.0093, -0.9855,  0.0000,  0.0000,  0.0000,  0.9762, -0.0093],
          [-0.0093, -0.9855,  0.0000,  0.0000,  0.0000,  0.9762, -0.0093],
          [-0.0093, -0.9855,  0.0000,  0.0000,  0.0000,  0.9762, -0.0093],
          [-0.0093, -0.9855,  0.0000,  0.0000,  0.0000,  0.9762, -0.0093],
          [-0.0093, -0.9855,  0.0000,  0.0000,  0.0000,  0.9762, -0.0093]]]],
       grad_fn=<ConvolutionBackward0>)

In [21]:
x = torch.tril(torch.ones(6, 8))
k = torch.tensor([[-1.,1.]])
y = corr2d(x,k)
x,y

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0.]]),
 tensor([[-1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0., -1.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0., -1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0., -1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0., -1.,  0.]]))

In [22]:
corr2d(x.T,k)

tensor([[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [23]:
corr2d(x, k.T)

tensor([[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.]])